# GIL(Global Interpreter Lock)
- Global Interpreter Lock
- 파이썬 인터프리터가 한 스레드만 하나의 바이트코드를 실행 시킬 수 있도록 해주는 Lock.
- 하나의 스레드에 모든 자원을 허락하고 그 후에는 Lock을 걸어 다른 스레드는 실행할 수 없게 막아버리는 것
- 즉 파이썬 쓰레드는 한번에 하나 밖에 동작을 못 함.

### Python 인터프리터란
- Python 인터프리터란, Python으로 작성된 코드를 한 줄씩 읽으면서 실행하는 프로그램(compile).
- 실행 프로그램의 구현체로 여러 가지가 있는데, 현재 Python 인터프리터의 표준 구현체로 받아들여지고 있는 것은 바로 **CPython**이다.
- CPython은 C 언어를 이용하여 구현한 Python 인터프리터.

In [1]:
# Interpreter
print("hello") # 1 -> C-compile : printf('hello') 
print("world") # 2 -> C-compile : printf('world')

hello
world


In [19]:
import sys

a=[1,2,3]
b=a

print( sys.getrefcount(a) ) # 현재 함수도 reference에 포함
del b
print( "after removing b : ", sys.getrefcount(a) )

3
after removing b :  2


### 비동기 처리
- [참조](/notebooks/머신러닝엔지니어/병렬처리/비동기%20처리.ipynb)

### GIL(Global Interpreter Lock)
- 전역 인터프리터 락
>In CPython, the global interpreter lock, or GIL, is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecodes at once. This lock is necessary mainly because CPython's memory management is not thread-safe.
- Python의 객체들에 대한 접근을 보호하는 일종의 뮤텍스(Mutex)로서, 여러 개의 쓰레드(멀티 코어)가 파이썬 코드를 동시에 실행하지 못하도록 하는 것.
- 즉, **한 프로세스 내에서, Python 인터프리터는 한 시점에 하나의 쓰레드에 의해서만 실행될 수 있음**
- 원래 멀티 코어라면 멀티 쓰레딩 시에 여러 개의 쓰레드가 여러 코어 상에서 병렬(Parallel) 실행될 수 있는데, Python에서는 그러한 병렬 실행이 불가능하다는 것
- 즉 멀티 쓰레딩-비동기가 아니라 **멀티 쓰레딩-동기**를 의미한다.

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FG0yFa%2Fbtq8KR1x1uD%2FVEj2oyOxwS1146fCrtqB7k%2Fimg.png' width='500px'></img>

### GIL 사용 하는 이유
- CPython이 메모리를 관리하는 방법이 thread-safe하지 않기 때문

### Race condition 문제

In [18]:
import threading

x = 0  # A shared value
def foo():
    global x
    for i in range(1000000):
        x += 1

def bar():
    global x
    for i in range(1000000):
        x -= 1

t1 = threading.Thread(target=foo)
t2 = threading.Thread(target=bar)
t1.start()
t2.start()

t1.join()
t2.join()  # Wait for completionprint(x)

print(x)

-156512


전역 변수 x에 두 개의 thread가 동시에 접근해서 각자의 작업을 하면서 어느 한 쪽의 작업 결과가 반영이 되지 않기 때문

### mutex
- Thread-safe한 코드를 만들기 위해서 사용하는 것 중 하나가 mutex (mutual exclusion)이다. 위에서 본 참사를 막기 위해서, 공유되는 메모리의 데이터를 여러 thread가 동시에 사용할 수 없도록 잠그는 일을 mutex가 맡는다.


### GIL이 있는데 멀티 쓰레드를 쓰는 이유
- 멀티 쓰레드를 쓰는 이유는! 각 쓰레드가 동작을 하되, IO block과 같은 대기하는 시간이 생길 때 다른 쓰레드가 동작을 함으로써 병렬처리가 가능

### CPU 연산을 병렬 처리 하는 법 (GIL 우회 방법)
- 일반적인 CPU 연산에 대하여, 병렬 처리를 하려면 크게 두 가지 방법이 있음.

####  멀티 프로세싱을 이용하는 방법
- 한 프로세스의 여러 쓰레드들은 서로 자원을 공유하지만, 여러 프로세스들은 각자 독자적인 메모리 공간을 가져서 서로 자원을 공유하지 않기 때문이다. 물론 자원을 공유하려면 할 수는 있지만, 별도의 처리가 필요하다. 다만, 멀티 프로세싱은 메모리를 더 필요로 하고 문맥 전환의 비용이 꽤 된다는 단점이 있다.
 
#### CPython이 아닌 다른 Python 인터프리터 구현체를 사용 방법
- 예를 들면 Jython 등이 있다. 그러나 흔히 사용하는 방법은 아니므로 권장하지 않는다.